In [1]:
import s4sim.hardware
c=s4sim.hardware.get_example()

In [2]:
import s4sim

In [3]:
git_hash = s4sim.__version__.split(".")[-1][1:]
git_hash

'e20d9ab'

The git commit hash is the last section after the dot of this version id after removing the first character.

In [4]:
c.data["telescopes"]["SAT1"]["fwhm"]

OrderedDict([('SAT_f030', 85.2),
             ('SAT_f040', 61.6),
             ('SAT_f085', 24.6),
             ('SAT_f145', 15.8),
             ('SAT_f095', 22.4),
             ('SAT_f155', 14.3),
             ('SAT_f220', 9.4),
             ('SAT_f280', 7.7)])

In [5]:
c.data["telescopes"].keys()

odict_keys(['LAT0', 'LAT1', 'LAT2', 'SAT1', 'SAT2', 'SAT3'])

In [6]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

odict_keys(['SAT_f030', 'SAT_f040', 'SAT_f085', 'SAT_f145', 'SAT_f095', 'SAT_f155', 'SAT_f220', 'SAT_f280'])

In [7]:
c.data["telescopes"]["SAT3"].keys()

odict_keys(['tubes', 'platescale', 'tubespace', 'fwhm'])

In [8]:
c.data["bands"].keys()

odict_keys(['SPLAT_f020', 'CHLAT_f030', 'CHLAT_f040', 'SPLAT_f030', 'SPLAT_f040', 'SAT_f030', 'SAT_f040', 'CHLAT_f090', 'CHLAT_f150', 'SPLAT_f090', 'SPLAT_f150', 'SAT_f085', 'SAT_f145', 'SAT_f095', 'SAT_f155', 'CHLAT_f220', 'CHLAT_f280', 'SPLAT_f220', 'SPLAT_f280', 'SAT_f220', 'SAT_f280'])

In [9]:
import numpy as np

In [10]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [11]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from `s4sim`, version {git_hash}"]

In [12]:
c.data["telescopes"]["LAT0"]["fwhm"]

OrderedDict([('SPLAT_f020', 11.2),
             ('CHLAT_f030', 7.8),
             ('SPLAT_f030', 9.1),
             ('CHLAT_f040', 5.3),
             ('SPLAT_f040', 6.2),
             ('CHLAT_f090', 2.1),
             ('SPLAT_f090', 2.5),
             ('CHLAT_f150', 1.3),
             ('SPLAT_f150', 1.5),
             ('CHLAT_f220', 0.95),
             ('SPLAT_f220', 1.1),
             ('CHLAT_f280', 0.83),
             ('SPLAT_f280', 1.0)])

In [13]:
s4 = []
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    tel = band.split("_")[0][-3:]
    assert tel in ["SAT", "LAT"]
    fwhm = c.data["telescopes"][tel + "1"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    d = OrderedDict(band=band)
    d["band"] = band
    d["telescope"] = tel
    if band.startswith("SP"):
        d["site"] = "Pole"
    elif band.startswith("CH"):
        d["site"] = "Chile"
    else:
        d["site"] = "Both"
    d["center_frequency"] = v["center"] * u.GHz
    d["fwhm"] = fwhm * u.arcmin
    # d["NET"] = v["NET"] * u.uK*u.s**.5
    d["nside"] = 4096 if tel =="LAT" else 512
    bandpass_table = QTable(dict(bandpass_frequency=np.linspace(v["low"], v["high"], 10) * u.GHz,
    bandpass_weight=np.ones(10, dtype=np.float)))
    bandpass_table.meta["comments"] = ipac_warning
    bandpass_table.write(f"../instrument_model/bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)
    s4.append(d)

## SPLAT_f020
Bandpass [GHz] low,center,high:  17.5 20.0 22.5
Beam [arcmin]:  11.2


/tmp/ipykernel_185846/3923528699.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bandpass_weight=np.ones(10, dtype=np.float)))


## CHLAT_f030
Bandpass [GHz] low,center,high:  21.5 24.75 28.0
Beam [arcmin]:  7.8
## CHLAT_f040
Bandpass [GHz] low,center,high:  28.0 36.5 45.0
Beam [arcmin]:  5.3
## SPLAT_f030
Bandpass [GHz] low,center,high:  21.5 24.75 28.0
Beam [arcmin]:  9.1
## SPLAT_f040
Bandpass [GHz] low,center,high:  28.0 36.5 45.0
Beam [arcmin]:  6.2
## SAT_f030
Bandpass [GHz] low,center,high:  21.5 24.75 28.0
Beam [arcmin]:  85.2
## SAT_f040
Bandpass [GHz] low,center,high:  28.0 36.5 45.0
Beam [arcmin]:  61.6
## CHLAT_f090
Bandpass [GHz] low,center,high:  77.0 91.5 106.0
Beam [arcmin]:  2.1
## CHLAT_f150
Bandpass [GHz] low,center,high:  128.0 148.5 169.0
Beam [arcmin]:  1.3
## SPLAT_f090
Bandpass [GHz] low,center,high:  77.0 91.5 106.0
Beam [arcmin]:  2.5
## SPLAT_f150
Bandpass [GHz] low,center,high:  128.0 148.5 169.0
Beam [arcmin]:  1.5
## SAT_f085
Bandpass [GHz] low,center,high:  74.8 85.0 95.2
Beam [arcmin]:  24.6
## SAT_f145
Bandpass [GHz] low,center,high:  129.1 145.0 161.0
Beam [arcmin]:  15.8
## SAT

/tmp/ipykernel_64022/3923528699.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bandpass_weight=np.ones(10, dtype=np.float)))


## Add reference to bandpasses

In [14]:
for ch in s4:
    ch["bandpass_file"] = f"bandpass_{ch['band']}.tbl"

In [15]:
len(s4)

21

In [16]:
%ls ../instrument_model/band* | wc -l

21


# Create the astropy Table

In [17]:
import astropy.units as u

In [18]:
from astropy.table import QTable, Table

In [19]:
t = QTable(rows=s4, masked=True, names=list(s4[1]))
t.add_index("band")

In [20]:
t.meta["comments"] = ipac_warning

In [21]:
t.sort(["telescope", "center_frequency"])

In [22]:
t.write("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac", overwrite=True)

In [23]:
c = QTable.read("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [24]:
c.add_index("band")

In [25]:
c

band,telescope,site,center_frequency,fwhm,nside,bandpass_file
,,,GHz,arcmin,,
str10,str3,str5,float64,float64,int64,str23
SPLAT_f020,LAT,Pole,20.0,11.2,4096,bandpass_SPLAT_f020.tbl
CHLAT_f030,LAT,Chile,24.75,7.8,4096,bandpass_CHLAT_f030.tbl
SPLAT_f030,LAT,Pole,24.75,9.1,4096,bandpass_SPLAT_f030.tbl
CHLAT_f040,LAT,Chile,36.5,5.3,4096,bandpass_CHLAT_f040.tbl
SPLAT_f040,LAT,Pole,36.5,6.2,4096,bandpass_SPLAT_f040.tbl
SPLAT_f090,LAT,Pole,91.5,2.5,4096,bandpass_SPLAT_f090.tbl
CHLAT_f090,LAT,Chile,91.5,2.1,4096,bandpass_CHLAT_f090.tbl
CHLAT_f150,LAT,Chile,148.5,1.3,4096,bandpass_CHLAT_f150.tbl


In [26]:
cat ../instrument_model/cmbs4_instrument_model.tbl

\ Text file in IPAC table format, read with astropy
\ from astropy.table import QTable
\ QTable.read('filename.tbl', format='ascii.ipac')
\ Instrument model exported from `s4sim`, version e20d9ab
|      band|telescope| site|center_frequency|  fwhm|nside|          bandpass_file|
|      char|     char| char|          double|double| long|                   char|
|          |         |     |             GHz|arcmin|     |                       |
|      null|     null| null|            null|  null| null|                   null|
 SPLAT_f020       LAT  Pole             20.0   11.2  4096 bandpass_SPLAT_f020.tbl 
 CHLAT_f030       LAT Chile            24.75    7.8  4096 bandpass_CHLAT_f030.tbl 
 SPLAT_f030       LAT  Pole            24.75    9.1  4096 bandpass_SPLAT_f030.tbl 
 CHLAT_f040       LAT Chile             36.5    5.3  4096 bandpass_CHLAT_f040.tbl 
 SPLAT_f040       LAT  Pole             36.5    6.2  4096 bandpass_SPLAT_f040.tbl 
 SPLAT_f090       LAT  Pole             91.5    2.5  4096

## Compare with the instrument model used to run the Chile LAT simulations

Check that center frequencies and fwhm are the same

In [27]:
lat_instrument_model = QTable.read("../../202102_design_tool_run/instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [28]:
lat_instrument_model.add_index("band")

In [29]:
lat_instrument_model = lat_instrument_model[lat_instrument_model["telescope"] == "LAT"]

In [30]:
chlat_instrument_model = lat_instrument_model[["P" not in b for b in lat_instrument_model["band"]]]

In [31]:
chlat_instrument_model

band,telescope,center_frequency,fwhm,detectors_per_tube,NET,chile_temp_sensitivity_DSR,chile_pol_sensitivity_DSR,chile_temp_sensitivity_PDB,chile_pol_sensitivity_PDB,pole_pol_sensitivity_DSR,pole_pol_sensitivity_PDB
,,GHz,arcmin,,s(1/2) uK,arcmin uK,arcmin uK,arcmin uK,arcmin uK,arcmin uK,arcmin uK
str6,str3,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64
LFL1,LAT,25.75,7.4,96,275.0,21.8,30.8,21.34,30.23,5.0,6.5
LFL2,LAT,38.75,5.1,96,215.0,12.4,17.6,11.67,16.53,4.5,4.15
MFL1,LAT,91.5,2.2,864,251.0,2.0,2.9,1.89,2.68,0.68,0.63
MFL2,LAT,148.5,1.4,864,280.0,2.0,2.8,2.09,2.96,0.96,0.59
HFL1,LAT,227.0,1.0,864,625.0,6.9,9.8,6.9,9.78,5.7,1.83
HFL2,LAT,285.5,0.9,864,1528.0,16.7,23.6,16.88,23.93,9.8,4.34


In [32]:
chlat_new_instrument_model = c[c["site"] == "Chile"]

In [33]:
chlat_new_instrument_model

band,telescope,site,center_frequency,fwhm,nside,bandpass_file
,,,GHz,arcmin,,
str10,str3,str5,float64,float64,int64,str23
CHLAT_f030,LAT,Chile,24.75,7.8,4096,bandpass_CHLAT_f030.tbl
CHLAT_f040,LAT,Chile,36.5,5.3,4096,bandpass_CHLAT_f040.tbl
CHLAT_f090,LAT,Chile,91.5,2.1,4096,bandpass_CHLAT_f090.tbl
CHLAT_f150,LAT,Chile,148.5,1.3,4096,bandpass_CHLAT_f150.tbl
CHLAT_f220,LAT,Chile,227.0,0.95,4096,bandpass_CHLAT_f220.tbl
CHLAT_f280,LAT,Chile,285.5,0.83,4096,bandpass_CHLAT_f280.tbl


In [34]:
chlat_instrument_model["fwhm"] == chlat_new_instrument_model["fwhm"]

array([False, False, False, False, False, False])

In [35]:
chlat_instrument_model["center_frequency"] == chlat_new_instrument_model["center_frequency"]

array([False, False,  True,  True,  True,  True])